In [ ]:
##Note that linear regression and Adaline are very similar. The only difference is that we apply a threshold function for converting the outputs from continuous targets for predictions.
###The derivative and training procedure are identical to Adaline though 


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
%matplotlib inline


### Load & Prepare Toy Dataset

In [2]:
df = pd.read_csv('linreg-data.csv' , index_col = 0)
df.tail()

,x1,x2,y
995,-0.942094,-0.835856,-22.324428
996,1.222445,-0.403177,-52.121493
997,-0.112466,-1.688230,-57.043196
998,-0.403459,-0.412272,-27.701833
999,0.021351,-0.499017,-9.804714


In [3]:
##Assign features and target 
X = torch.tensor(df[['x1' , 'x2']].values , dtype = torch.float)
y = torch.tensor(df['y'].values , dtype = torch.float)


##Shuffling & train/test split 
#set seed 
torch.manual_seed(123)
#
shuffle_idx = torch.randperm(y.size(0) , dtype = torch.long)

X,y = X[shuffle_idx] , y[shuffle_idx] 
percent70 = int(shuffle_idx.size(0) * 0.7)

X_train , X_test = X[shuffle_idx[ :percent70]] , X[shuffle_idx[percent70:]]
y_train , y_test = y[shuffle_idx[:percent70]] , y[shuffle_idx[percent70:]]

#Normalizing data 
mu,sigma = X_train.mean(dim = 0) , X_train.std(dim = 0)
X_train = (X_train - mu)/sigma
X_test = (X_test - mu)/sigma 




### Implementing Linear Regression Model

In [7]:
class LinearRegression1():
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(num_features, 1, 
                                   dtype=torch.float)
        self.bias = torch.zeros(1, dtype=torch.float)

    def forward(self, x):
        netinputs = torch.add(torch.mm(x, self.weights), self.bias)
        activations = netinputs
        return activations.view(-1)
        
    def backward(self, x, yhat, y):  
        
        grad_loss_yhat = 2*(yhat - y)
        
        grad_yhat_weights = x
        grad_yhat_bias = 1.
        
        # Chain rule: inner times outer
        grad_loss_weights =  torch.mm(grad_yhat_weights.t(),
                                         grad_loss_yhat.view(-1, 1)) / y.size(0)

        grad_loss_bias = torch.sum(grad_yhat_bias*grad_loss_yhat) / y.size(0)
        
        # return negative gradient
        return (-1)*grad_loss_weights, (-1)*grad_loss_bias

### Defining training and evaluation functions 

In [8]:
########################
##Training and wrappers
########################
def loss(yhat,y):
    return torch.mean((yhat -y) **2)

def train( model , x , y , num_epochs, learning_rate = 0.01):
    cost = []
    for e in range(num_epochs):
        
        ##compute outputs 
        yhat = model.forward(x)
        
        ###compute gradients 
        negative_grad_w , negative_grad_b = model.backward(x , yhat , y)
        
        ###Updates weights 
        model.weights += learning_rate*negative_grad_w 
        model.bias += learning_rate*negative_grad_b
        
        ###logging 
        
        yhat = model.forward(x)
        curr_loss = loss(yhat , y)
        print('Epoch: %03d' %(e+1) , end = '')
        print(' | MSE: %.5f' % curr_loss)
        cost.append(curr_loss)
    return cost
        

### Train Linear Regression Model 

In [9]:
model = LinearRegression1()
cost  = train(model , 
             X_train , y_train , 
             num_epochs = 100 , 
             learning_rate = 0.05)


TypeError: LinearRegression1.__init__() missing 1 required positional argument: 'num_features'

In [24]:
train_pred = model.forward(X_train)
test_pred = model.forward(X_test)

print('Train MSE: %.5f' % loss(train_pred, y_train))
print('Test MSE: %.5f' % loss(test_pred, y_test))

AttributeError: 'LinearRegression1' object has no attribute 'weights'